In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers
from transformers import BertModel, BertTokenizer
import joblib

# Load data comment 

In [8]:
df = pd.read_csv('data_crawler.csv', delimiter='\t', header=None)
print(df.shape)
# get all rows
print(df[0]) 

(133, 1)
0      18/4 Lazada giao nhưng 19/4 tôi mới khui hộp m...
1      Đã nhận được hàng, máy đẹp, giao hàng nhanh. C...
2      Tôi mua sản phẩm samsung Galaxy M10. Máy bị lỗ...
3      Đặt Samsung Galaxy M10,giao hàng ngoài hộp M10...
4      Hàng đẹp xài ngon so với giá tiền, nên mua cho...
                             ...                        
128               Mua 25/2 mở ra thấy bị lỗi sơn viền ,1
129       Hàng ok nhưng thấy vỏ hộp có vết mực bi đánh,1
130                Sản_phẩm đúng như mô_tả Nguyên seal,0
131    Mọi thứ quay mồng mồng mà không làm được gì. G...
132    mới mua về mà có cuộc gọi đến đều không bấm đư...
Name: 0, Length: 133, dtype: object


# Load Pretrain model BERT

In [6]:
'''
Load pretrain model/ tokenizers
'''
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

#encode lines
tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens = True)))
print('encode',tokenized[1])
# decode
print('decode',tokenizer.decode(tokenized[1]))

c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you 

encode [101, 1102, 2050, 18699, 2319, 1102, 19098, 2278, 6865, 1010, 2089, 1102, 13699, 1010, 27699, 2080, 6865, 18699, 2319, 2232, 1012, 11503, 2006, 14841, 3211, 1010, 1014, 102]
decode [CLS] đa nhan đuoc hang, may đep, giao hang nhanh. cam on tiki, 0 [SEP]


# Fine tuning model and save model

In [9]:
#get all label 
labels = np.zeros(len(df[0]))
for i in range(len(df[0])):
    labels[i] = df[0][i][-1]
print('labels shape:', labels.shape)

# get lenght max of tokenized
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print('max len:', max_len)

# if lenght of tokenized not equal max_len , so padding value 0
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])
print('padded:', padded[1])
print('len padded:', padded.shape)

#get attention mask ( 0: not has word, 1: has word)
attention_mask = np.where(padded ==0, 0,1)
print('attention mask:', attention_mask[1])

# Convert input to tensor
padded = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)


# Train model
with torch.no_grad():
    last_hidden_states = model(padded, attention_mask =attention_mask)
#     print('last hidden states:', last_hidden_states)

features = last_hidden_states[0][:,0,:].numpy()
print('features:', features)

X_train, X_test, y_train, y_test = train_test_split(features, labels)

cl = LogisticRegression()
cl.fit(X_train, y_train)

# Save model
joblib.dump(cl, 'save_model.pkl')
sc = cl.score(X_test, y_test)
print('score:', sc)


labels shape: (133,)
max len: 138
padded: [  101  1102  2050 18699  2319  1102 19098  2278  6865  1010  2089  1102
 13699  1010 27699  2080  6865 18699  2319  2232  1012 11503  2006 14841
  3211  1010  1014   102     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
len padded: (133, 138)
attention mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0
 0 